<a href="https://colab.research.google.com/github/PratikshaPP/Sage-Project/blob/master/Sage_project_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install arrow

     |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
#Imports
import requests
from bs4 import BeautifulSoup
import datetime
import arrow

In [0]:
#Create a date:value dictionary and initialize it
date_dict = {}
start = datetime.datetime.strptime(arrow.get("01 January 2019","DD MMMM YYYY").isoformat()[:19],"%Y-%m-%dT%H:%M:%S")
end = datetime.datetime.strptime(arrow.get("31 December 2019","DD MMMM YYYY").isoformat()[0:19],"%Y-%m-%dT%H:%M:%S")

delta = end - start
fmt = "YYYY-MM-DD HH:mm:ss"
for i in range(delta.days + 1):
    date_dict[str(arrow.get(str(start + datetime.timedelta(days=i)),fmt))] = 0

In [0]:
URL = "https://en.wikipedia.org/wiki/2019_in_spaceflight#Orbital_launches"

In [0]:
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

In [0]:
tb = soup.find("table", attrs={"class": "wikitable"})

In [0]:
rows = tb.findAll("tr")

In [0]:
# Scraping logic
for row in rows[4:]:
    try:
        payloads = row.findAll("td")
        
        if len(payloads) == 5:
            date = payloads[0].span.text
            #print(date)
        else:

            i = 1
            success = 0
            while(len(payloads)==6):
                if(payloads[i].get_text(strip=True)[:11].lower() == "operational" or payloads[i].get_text(strip=True).lower()[:10] == "successful"or payloads[i].get_text(strip=True).lower()[:8] == "en route"):
                  success = 1
                  break
                i = i+1

            if(success == 1 and len(date)!=0):
                key = arrow.get(date+" 2019","D MMMM YYYY").isoformat()
                date_dict[key] = date_dict[key]+success
              
                date = ""

    except:
        pass
    

In [0]:
#If you are using google colab, run this before executing the next block of code
import os
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Provide the path of the csv output file in your local/colab
csv_file = "/content/drive/My Drive/Colab Notebooks/output.csv"

with open(csv_file, 'w') as f:
    f.write("%s, %s\n" % ("date","value"))
    for key in date_dict.keys():
        f.write("%s, %s\n" % (key, str(date_dict[key])))
